In [1]:
using Pkg; Pkg.activate("..")
using Revise
using Distributions
import StatsBase.weights
using Random
using RCall
using ProgressBars
using GraphFusedLasso
using FileIO
using DataFrames, CSV
using Printf

R"library('tidyverse')"

# Random.seed!(418916);

┌ Warning: RCall.jl: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
│ ✔ ggplot2 3.1.0       ✔ purrr   0.3.2  
│ ✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
│ ✔ tidyr   0.8.3       ✔ stringr 1.4.0  
│ ✔ readr   1.3.1       ✔ forcats 0.4.0  
│ ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
│ ✖ dplyr::filter() masks stats::filter()
│ ✖ dplyr::lag()    masks stats::lag()
└ @ RCall /home/mauriciogtec/.julia/packages/RCall/iojZI/src/io.jl:113


In [50]:
function generate_task_dynamic(task, N)
#     cuts = [N ÷ 3, 2(N ÷ 3)]
    cuts = sort(sample(2:N-1, 2, replace=false))
    x1 = 1:cuts[1]
    x2 = (cuts[1] + 1):cuts[2]
    x3 = (cuts[2] + 1):N
    x = [x1; x2; x3]
    
    values1 = 2.0(rand(Uniform(), 4) .- 0.5)

    if task == "smooth"
        y1 = values1[1] .+ (values1[2] - values1[1]) .* (x1 ./ cuts[1])
        y2 = values1[2] .+ (values1[3] - values1[2]) .* (x2 .- cuts[1]) ./ (cuts[2] - cuts[1])
        y3 = values1[3] .+ (values1[4] - values1[3]) .* (x3 .- cuts[2]) ./ (N - cuts[2])
    elseif task == "constant"
        y1 = fill(values1[1], cuts[1])
        y2 = fill(values1[2], cuts[2] - cuts[1])
        y3 = fill(values1[3], N - cuts[2])
    elseif task == "mixed" || task == "mixed+outliers"
        y1 = values1[1] .+ Int(rand() < 0.5) .* (values1[2] - values1[1]) .* (x1 ./ cuts[1])
        y2 = values1[2] .+ Int(rand() < 0.5) .* (values1[3] - values1[2]) .* (x2 .- cuts[1]) ./ (cuts[2] - cuts[1])
        y3 = values1[3] .+ Int(rand() < 0.5) .* (values1[4] - values1[3]) .* (x3 .- cuts[2]) ./ (N - cuts[2])
    else
        throw(ArgumentError)
    end
    μ = [y1; y2; y3]
    return μ
end

function generate_task_dynamic(task, N, pmiss, σ)  
    μ1 = generate_mu_task(task, N)
    μ2 = generate_mu_task(task, N)
    
    evalpts = collect(range(-2.5, 2.5, length=100))
    dmodels = [MixtureModel([Normal(μ1[i], σ), Normal(μ2[i], σ)])
                 for i in 1:N]
    devals = [pdf.(d, evalpts) for d in dmodels];
    ndata = [sample([0, 10], weights([pmiss, 1.0 - pmiss])) for d in dmodels]
    y = [rand(d, n) for (d, n) in zip(dmodels, ndata)]
    
    if task == "mixed+ouliers"
        Nobs = sum([1 for n in ndata if n > 0])
        K = floor(Nobs * 0.5)
        idx = sample([i for (i, n) in enumerate(ndata) if n > 0], K, replace=false)
        for i in idx
            j = rand(1:length(y[i]))
            y[i][j] += rand([-1, 1]) * 5.0
        end
    end
                    
    ptr = collect(1:N)
    brks = [1, N + 1]
    
    return Dict("evalpts" => evalpts,
                "dmodels" => dmodels,
                "devals" => devals,
                "y" => y,
                "ndata" => ndata,
                "mean1" => μ1,
                "mean2" => μ2,
                "values1" => values1,
                "values2" => values2,
                "ptr" => ptr,
                "brks" => brks)
end

function generate_bivariate_task(task1, task2, N, pmiss, σ=0.3, outliers=false)
    μ1s = generate_task_dynamic(task1, N)
    μ2s = generate_task_dynamic(task1, N)
    ts =  generate_task_dynamic(task2, N)
    
    evalpts = range(-2.5, 2.5, length=100)
    
    μs = [(t + μ1, t + μ2) for (μ1, μ2) in zip(μ1s, μ2s), t in ts]
    dmodels = [MixtureModel([Normal(μ1, σ), Normal(μ2, σ)]) for (μ1, μ2) in μs]

    devals = [pdf.(d, evalpts) for d in dmodels];
    ndata = [sample([0, 10], weights([pmiss, 1.0 - pmiss])) for d in dmodels]
    y = [rand(d, n) for (d, n) in zip(dmodels, ndata)]
    
    if outliers
        Nobs = sum([1 for n in ndata if n > 0])
        K = floor(Nobs * 0.5)
        idx = sample([i for (i, n) in enumerate(ndata) if n > 0], K, replace=false)
        for i in idx
            j = rand(1:length(y[i]))
            y[i][j] += rand([-1, 1]) * 5.0
        end
    end
                    
    # make matrix pts
    xrange = collect(1:N)
    # temporal
    ptr = []
    brks = [1]
    for i in 1:N
        append!(ptr, xrange .+ (i - 1) * N)
        push!(brks, brks[end] + N)
    end
    istemporal = fill(true, N^2)
    # spatial
    xrange = [(i - 1) * N + 1 for i in 1:N]
    for i in 1:N
        append!(ptr, xrange .+ (i - 1))
        push!(brks, brks[end] + N)
    end
    append!(istemporal, fill(false, N^2))
    
    return Dict("evalpts" => evalpts,
                "dmodels" => dmodels,
                "devals" => devals,
                "y" => y,
                "ndata" => ndata,
                "mean1" => μ1s,
                "mean2" => μ2s,
                "t" => ts,
                "means" => μs,
                "ptr" => ptr,
                "brks" => brks,
                "istemporal" => istemporal)
end

# function for cross-validation fit

function generate_cvsets(y, nsplits)
    # make the cv splits
    N = length(y)
    cvsets = [Set{Int}() for i in 1:nsplits]
    iobs = shuffle([i for (i, yi) in enumerate(y) if !isempty(yi)])
    Nobs = length(iobs)
    splitsize = Nobs ÷ nsplits
    for k in 1:nsplits
        for i in ((k - 1) * splitsize + 1):(k * splitsize)
            push!(cvsets[k], iobs[i])
        end
    end
    return cvsets
end
               
                
function fit(ytrain, ptr, brks, λ1, λ2)
    N = length(ytrain)
    lambdasl1 = fill(λ1, N)
    lambdasl2 = fill(λ2, N)

    # create the tree
    M  = 33
    splits = collect(range(-2.5, 2.5, length=M))
    tree = DensityTree(splits)
    bins2counts = Dict()
    for (j, (li, ui)) in enumerate([tree.bins; [(i, i+1) for i in 1:M-1]])
        lower = splits[li]
        upper = splits[ui]
        k = [sum(lower .< yi .< upper) for yi in ytrain]
        bins2counts[(li, ui)] = k
    end
            
    # fit binomial model in each tree
    beta = zeros(N, M - 2)
    for j in 1:M - 2
        li, ui = tree.bins[j]
        mi = (ui + li) ÷ 2 
        parent_counts = bins2counts[(li, ui)] .+ 0.1
        left_counts = bins2counts[(li, mi)]  .+ 0.05

        lambdasl1 = fill(λ1, N)
        lambdasl2 = fill(λ2, N)
        model = BinomialEnet(
            ptr, brks, lambdasl1, lambdasl2;
            abstol=0.0, reltol=1e-4)
        fit!(model, left_counts, parent_counts; steps=1000)

        beta[:, j] = model.beta
    end
    tree.beta = beta
    return tree
end


function fit2(ytrain, ptr, brks, λ1, λ2, η1, η2)
    N = length(ytrain)
    lambdasl1 = fill(λ1, N)
    lambdasl2 = fill(λ2, N)

    # create the tree
    M  = 33
    splits = collect(range(-2.5, 2.5, length=M))
    tree = DensityTree(splits)
    bins2counts = Dict()
    for (j, (li, ui)) in enumerate([tree.bins; [(i, i+1) for i in 1:M-1]])
        lower = splits[li]
        upper = splits[ui]
        k = [sum(lower .< yi .< upper) for yi in ytrain]
        bins2counts[(li, ui)] = k
    end
            
    # fit binomial model in each tree
    beta = zeros(N, M - 2)
    for j in 1:M - 2
        li, ui = tree.bins[j]
        mi = (ui + li) ÷ 2 
        parent_counts = bins2counts[(li, ui)] .+ 0.1
        left_counts = bins2counts[(li, mi)]  .+ 0.05

        lambdasl1 = fill(λ1, N)
        lambdasl2 = fill(λ2, N)
        model = BinomialEnet(
            ptr, brks, lambdasl1, lambdasl2;
            abstol=0.0, reltol=1e-4)
        fit!(model, left_counts, parent_counts; steps=1000)

        beta[:, j] = model.beta
    end
    tree.beta = beta
    return tree
end

                
function integrated_squared_error(d1, d2, evalpts)
    f = (d1 .- d2) .^ 2
    δ = evalpts[2] .- evalpts[1]
    return δ * (0.5f[1] + 0.5f[2] + sum(f[2:end-1]))
end
         
function integrated_abs_error(d1, d2, evalpts)
    f = abs.(d1 .- d2)
    δ = evalpts[2] .- evalpts[1]
    return δ * (0.5f[1] + 0.5f[2] + sum(f[2:end-1]))
end
         
                
function cv_fit(y, evalpts, ptr, brks, lambdas, cvsets, devals)
    # for each cv split get the mse error
    N = length(y)
    nsplits = length(cvsets)
    nlambdas = length(lambdas)
    test_likelihood = zeros(nlambdas)
                    
    # prepare the tree structure and the bint 
    for (k, (λ1, λ2)) in enumerate(lambdas)
        for i in 1:nsplits
            # get the cv vector with missing data
            ytrain = [j in cvsets[i] ? Float64[] : yi
                      for (j, yi) in enumerate(y)]

            tree = fit(ytrain, ptr, brks, λ1, λ2)               
                            
            # compute the out-of-sample likelihood
            Ntest = 0.0
            likelihood = 0.0
            for j in collect(cvsets[i])
                test_eval = y[j]
                Ntest += length(test_eval)
                likelihood += sum(predict(tree, sort(test_eval))[j])
            end
            likelihood /= Ntest
                             
            test_likelihood[k] += likelihood / nsplits
            dhats = predict(tree, evalpts)
        end
    end

    # now choose the best lambdas
    best_lambdas = lambdas[argmax(test_likelihood)]
    best_likelihood = maximum(test_likelihood)

    # train for best params
    best_tree = fit(y, ptr, brks, best_lambdas[1], best_lambdas[2])
    densities = predict(best_tree, evalpts)
    rmise = 0.0
    miae = 0.0
    for (d1, d2) in zip(devals, densities)
        rmise += integrated_squared_error(d1, d2, evalpts)
        miae += integrated_abs_error(d1, d2, evalpts)
    end
    rmise = sqrt(rmise / N)
    miae = miae / N

    return Dict("best_lambdas" => best_lambdas,
                "cv_likelihood" => best_likelihood,
                "densities" => densities,
                "rmise" => rmise,
                "miae" => miae)
end
            
function get_lambdas(method)
    lambdas_dict = Dict(
        "fl" => [(l, 1e-12) for l in range(1e-12, 3.0, length=25)],
        "kal" => [(1e-12, l) for l in range(1e-12, 30.0, length=25)],
        "enet" => [(l1, l2) for l1 in range(1e-12, 2.5, length=10)
                            for l2 in range(1e-12, 25.0, length=10)]) 
    return lambdas_dict[method]
end
            
function run_benchmarks(N, σ, pmiss;
                        nsims=100,
                        nsplits=5,
                        tasks=("constant", "smooth", "mixed", "mixed+outliers"))

    experiment_results = []
    for task in tasks
        data = [generate_data(task, N, pmiss, σ) for _ in 1:nsims]
        for method in ("fl", "kal", "enet")
            println("Running task $task for method $method")
            lambdas = get_lambdas(method)
            likelihood = zeros(nsims)
            rmise = zeros(nsims)
            miae = zeros(nsims)
            for (l, D) in ProgressBar(enumerate(data))
                y = D["y"]
                evalpts = D["evalpts"]
                ndata = D["ndata"]
                devals = D["devals"]
                ptr = D["ptr"]
                brks = D["brks"]
                cvsets = generate_cvsets(y, nsplits)

                results = cv_fit(y, evalpts, ptr, brks, lambdas, cvsets, devals)

                new_result = Dict(
                    :experiment => l,
                    :task => task,
                    :method => method,
                    :likelihood => results["cv_likelihood"],
                    :rmise => results["rmise"],
                    :miae => results["miae"])
                push!(experiment_results, new_result)
            end
        end
    end
    return experiment_results
end

run_benchmarks (generic function with 1 method)

In [51]:
experiment = generate_bivariate_task("smooth", "smooth", 30, 0.1)
experiment["means"]

30×30 Array{Tuple{Float64,Float64},2}:
 (0.236837, -0.417602)     (0.395158, -0.259281)    …  (1.11343, 0.458987)   
 (0.244942, -0.355344)     (0.403263, -0.197023)       (1.12153, 0.521245)   
 (0.253047, -0.293086)     (0.411368, -0.134765)       (1.12964, 0.583503)   
 (0.261152, -0.230828)     (0.419473, -0.0725066)      (1.13774, 0.645761)   
 (0.269257, -0.16857)      (0.427578, -0.0102485)      (1.14585, 0.708019)   
 (0.277362, -0.106311)     (0.435683, 0.0520096)    …  (1.15395, 0.770278)   
 (0.285467, -0.128952)     (0.443788, 0.0293695)       (1.16206, 0.747637)   
 (0.261493, -0.151592)     (0.419815, 0.00672937)      (1.13808, 0.724997)   
 (0.23752, -0.174232)      (0.395841, -0.0159108)      (1.11411, 0.702357)   
 (0.213547, -0.196872)     (0.371868, -0.0385509)      (1.09014, 0.679717)   
 (0.189574, -0.219512)     (0.347895, -0.061191)    …  (1.06616, 0.657077)   
 (0.165601, -0.242152)     (0.323922, -0.0838312)      (1.04219, 0.634437)   
 (0.141627, -0.264792)   

In [22]:
N = 150
σ = 0.3
pmiss = 0.8
nsims = 50
tasks = ("smooth", "constant", "mixed", "mixed+outliers")

experiment_results = run_benchmarks(N, σ, pmiss, nsims=nsims,tasks=tasks)

Running task smooth for method fl
100.00%┣██████████████████████████████████████████████████████████████┫ 50/50 07:50<00:00, 0.10 it/s]
Running task smooth for method kal
100.00%┣██████████████████████████████████████████████████████████████┫ 50/50 07:50<00:00, 0.10 it/s]
Running task smooth for method enet
100.00%┣██████████████████████████████████████████████████████████████┫ 50/50 54:50<00:00, 0.01 it/s]
Running task constant for method fl
100.00%┣██████████████████████████████████████████████████████████████┫ 50/50 08:18<00:00, 0.10 it/s]
Running task constant for method kal
100.00%┣██████████████████████████████████████████████████████████████┫ 50/50 08:13<00:00, 0.10 it/s]
Running task constant for method enet
100.00%┣██████████████████████████████████████████████████████████████┫ 50/50 58:05<00:00, 0.01 it/s]
Running task mixed for method fl
100.00%┣██████████████████████████████████████████████████████████████┫ 50/50 08:04<00:00, 0.10 it/s]
Running task mixed for method kal
100

600-element Array{Any,1}:
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.287274,:task=>"smooth",:likelihood=>0.52963,:experiment=>1,:miae=>0.403404)            
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.365551,:task=>"smooth",:likelihood=>0.674995,:experiment=>2,:miae=>0.497785)           
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.315245,:task=>"smooth",:likelihood=>0.648937,:experiment=>3,:miae=>0.392976)           
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.259446,:task=>"smooth",:likelihood=>0.543999,:experiment=>4,:miae=>0.37407)            
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.320425,:task=>"smooth",:likelihood=>0.560123,:experiment=>5,:miae=>0.411986)           
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.340007,:task=>"smooth",:likelihood=>0.617697,:experiment=>6,:miae=>0.433046)           
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.36282,:task=>"smooth",:likelihood=>0.490645,:experiment=>7,:miae=>0.493204)            
 Dict{Symbol,Any}(:method=>"fl",:rmise=>0.263022,:task=>"smooth"

In [23]:
df = DataFrame(experiment = Int[],
               task=String[],
               method=String[],
               likelihood=Float64[],
               rmise=Float64[],
               miae=Float64[])
for record in experiment_results
    push!(df, record)
end

In [24]:
head(df, 10)

,experiment,task,method,likelihood,rmise,miae
,Int64,String,String,Float64,Float64,Float64
1,1,smooth,fl,0.52963,0.287274,0.403404
2,2,smooth,fl,0.674995,0.365551,0.497785
3,3,smooth,fl,0.648937,0.315245,0.392976
4,4,smooth,fl,0.543999,0.259446,0.37407
5,5,smooth,fl,0.560123,0.320425,0.411986
6,6,smooth,fl,0.617697,0.340007,0.433046
7,7,smooth,fl,0.490645,0.36282,0.493204
8,8,smooth,fl,0.525978,0.263022,0.348088
9,9,smooth,fl,0.664618,0.263926,0.350197


In [25]:
R"""
df = $df %>% 
    group_by(task, method) %>%
    summarize(likelihood_mean = mean(rmise))
print(df)
"""

# A tibble: 12 x 3
# Groups:   task [4]
   task           method likelihood_mean
   <chr>          <chr>            <dbl>
 1 constant       enet             0.309
 2 constant       fl               0.302
 3 constant       kal              0.312
 4 mixed          enet             0.309
 5 mixed          fl               0.309
 6 mixed          kal              0.312
 7 mixed+outliers enet             0.313
 8 mixed+outliers fl               0.313
 9 mixed+outliers kal              0.314
10 smooth         enet             0.311
11 smooth         fl               0.317
12 smooth         kal              0.312


RObject{VecSxp}
# A tibble: 12 x 3
# Groups:   task [4]
   task           method likelihood_mean
   <chr>          <chr>            <dbl>
 1 constant       enet             0.309
 2 constant       fl               0.302
 3 constant       kal              0.312
 4 mixed          enet             0.309
 5 mixed          fl               0.309
 6 mixed          kal              0.312
 7 mixed+outliers enet             0.313
 8 mixed+outliers fl               0.313
 9 mixed+outliers kal              0.314
10 smooth         enet             0.311
11 smooth         fl               0.317
12 smooth         kal              0.312


In [26]:
CSV.write("benchmarks-results-3.csv", df)

"benchmarks-results-3.csv"